In [1]:
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [2]:
# ===============================
# ⚙️ 参数配置
# ===============================
birds = [
    "Black-footed Albatross",
    "Laysan Albatross",
    "Sooty_Albatross",
    "Groove_billed_Ani",
    "Crested_Auklet",
    "Least_Auklet",
    "Parakeet_Auklet",
    "Rhinoceros_Auklet",
    "Brewer_Blackbird",
    "Red_winged_Blackbird"
]

OUTPUT_DIR = "dataset"
LIMIT_PER_BIRD = 600
PER_PAGE = 50
SLEEP_TIME = 1.0
MAX_THREADS = 8     # 并行下载线程数

In [3]:
# ===============================
# 🌐 爬取 Bing 图片 URL（翻页）
# ===============================
def fetch_image_urls(query, limit=600, per_page=50, session=None, sleep_time=1.0):
    if session is None:
        session = requests.Session()

    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/114.0.0.0 Safari/537.36"
        )
    }

    urls = []
    offset = 0
    page = 1

    while len(urls) < limit:
        print(f"📄 [{query}] 第 {page} 页 (offset={offset})")
        url = f"https://www.bing.com/images/search?q={query}&form=HDRSC2&first={offset}"
        resp = session.get(url, headers=headers, timeout=10)

        if resp.status_code != 200:
            print(f"⚠️ 第 {page} 页请求失败: 状态码 {resp.status_code}")
            break

        soup = BeautifulSoup(resp.text, "html.parser")
        img_elements = soup.find_all("img")

        new_urls = []
        for img in img_elements:
            src = img.get("src") or img.get("data-src")
            if src and src.startswith("https://") and src not in urls:
                new_urls.append(src)

        if not new_urls:
            print("❌ 没有更多图片，提前结束。")
            break

        urls.extend(new_urls)
        print(f"✅ 第 {page} 页新增 {len(new_urls)} 张，累计 {len(urls)} 张。")

        offset += per_page
        page += 1
        time.sleep(sleep_time)

    print(f"📸 [{query}] 共抓取 {len(urls)} 张（目标 {limit}）\n")
    return urls[:limit]

In [4]:
# ===============================
# 💾 并行下载图片（带进度条）
# ===============================
def download_single_image(url, folder, idx):
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        ext = os.path.splitext(urlparse(url).path)[1]
        if ext.lower() not in [".jpg", ".jpeg", ".png"]:
            ext = ".jpg"
        fname = os.path.join(folder, f"{idx:04d}{ext}")
        with open(fname, "wb") as f:
            f.write(resp.content)
        return True
    except Exception:
        return False


def download_images_parallel(urls, folder, max_threads=8):
    os.makedirs(folder, exist_ok=True)
    success = 0

    with ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [
            executor.submit(download_single_image, url, folder, i)
            for i, url in enumerate(urls, start=1)
        ]
        for _ in tqdm(as_completed(futures), total=len(futures), desc=f"📥 {os.path.basename(folder)}", unit="img"):
            success += 1

    print(f"✅ {os.path.basename(folder)} 下载完成，共 {success} 张图片。")

In [5]:
# ===============================
# 🚀 主程序：并行处理多个类别
# ===============================
start_time = time.time()
print("🚀 开始批量爬取鸟类图片...\n")

for bird in birds:
    folder = os.path.join(OUTPUT_DIR, bird.replace(" ", "_"))
    print(f"==============================")
    print(f"🦉 开始爬取：{bird}")
    print(f"==============================")
    urls = fetch_image_urls(bird, limit=LIMIT_PER_BIRD, per_page=PER_PAGE, sleep_time=SLEEP_TIME)
    download_images_parallel(urls, folder, max_threads=MAX_THREADS)

print(f"\n🎉 所有鸟类下载完成！总耗时 {time.time() - start_time:.2f} 秒")

🚀 开始批量爬取鸟类图片...

🦉 开始爬取：Black-footed Albatross
📄 [Black-footed Albatross] 第 1 页 (offset=0)
✅ 第 1 页新增 40 张，累计 40 张。
📄 [Black-footed Albatross] 第 2 页 (offset=50)
✅ 第 2 页新增 38 张，累计 78 张。
📄 [Black-footed Albatross] 第 3 页 (offset=100)
✅ 第 3 页新增 31 张，累计 109 张。
📄 [Black-footed Albatross] 第 4 页 (offset=150)
✅ 第 4 页新增 38 张，累计 147 张。
📄 [Black-footed Albatross] 第 5 页 (offset=200)
✅ 第 5 页新增 31 张，累计 178 张。
📄 [Black-footed Albatross] 第 6 页 (offset=250)
✅ 第 6 页新增 28 张，累计 206 张。
📄 [Black-footed Albatross] 第 7 页 (offset=300)
✅ 第 7 页新增 26 张，累计 232 张。
📄 [Black-footed Albatross] 第 8 页 (offset=350)
✅ 第 8 页新增 26 张，累计 258 张。
📄 [Black-footed Albatross] 第 9 页 (offset=400)
✅ 第 9 页新增 17 张，累计 275 张。
📄 [Black-footed Albatross] 第 10 页 (offset=450)
✅ 第 10 页新增 9 张，累计 284 张。
📄 [Black-footed Albatross] 第 11 页 (offset=500)
❌ 没有更多图片，提前结束。
📸 [Black-footed Albatross] 共抓取 284 张（目标 600）



📥 Black-footed_Albatross: 100%|████████████████████████████████████████████████████| 284/284 [00:44<00:00,  6.32img/s]


✅ Black-footed_Albatross 下载完成，共 284 张图片。
🦉 开始爬取：Laysan Albatross
📄 [Laysan Albatross] 第 1 页 (offset=0)
✅ 第 1 页新增 40 张，累计 40 张。
📄 [Laysan Albatross] 第 2 页 (offset=50)
✅ 第 2 页新增 38 张，累计 78 张。
📄 [Laysan Albatross] 第 3 页 (offset=100)
✅ 第 3 页新增 36 张，累计 114 张。
📄 [Laysan Albatross] 第 4 页 (offset=150)
✅ 第 4 页新增 31 张，累计 145 张。
📄 [Laysan Albatross] 第 5 页 (offset=200)
✅ 第 5 页新增 31 张，累计 176 张。
📄 [Laysan Albatross] 第 6 页 (offset=250)
✅ 第 6 页新增 32 张，累计 208 张。
📄 [Laysan Albatross] 第 7 页 (offset=300)
✅ 第 7 页新增 34 张，累计 242 张。
📄 [Laysan Albatross] 第 8 页 (offset=350)
✅ 第 8 页新增 32 张，累计 274 张。
📄 [Laysan Albatross] 第 9 页 (offset=400)
✅ 第 9 页新增 33 张，累计 307 张。
📄 [Laysan Albatross] 第 10 页 (offset=450)
✅ 第 10 页新增 28 张，累计 335 张。
📄 [Laysan Albatross] 第 11 页 (offset=500)
✅ 第 11 页新增 14 张，累计 349 张。
📄 [Laysan Albatross] 第 12 页 (offset=550)
✅ 第 12 页新增 8 张，累计 357 张。
📄 [Laysan Albatross] 第 13 页 (offset=600)
❌ 没有更多图片，提前结束。
📸 [Laysan Albatross] 共抓取 357 张（目标 600）



📥 Laysan_Albatross: 100%|██████████████████████████████████████████████████████████| 357/357 [00:51<00:00,  6.89img/s]


✅ Laysan_Albatross 下载完成，共 357 张图片。
🦉 开始爬取：Sooty_Albatross
📄 [Sooty_Albatross] 第 1 页 (offset=0)
✅ 第 1 页新增 39 张，累计 39 张。
📄 [Sooty_Albatross] 第 2 页 (offset=50)
✅ 第 2 页新增 58 张，累计 97 张。
📄 [Sooty_Albatross] 第 3 页 (offset=100)
✅ 第 3 页新增 37 张，累计 134 张。
📄 [Sooty_Albatross] 第 4 页 (offset=150)
✅ 第 4 页新增 37 张，累计 171 张。
📄 [Sooty_Albatross] 第 5 页 (offset=200)
✅ 第 5 页新增 35 张，累计 206 张。
📄 [Sooty_Albatross] 第 6 页 (offset=250)
✅ 第 6 页新增 38 张，累计 244 张。
📄 [Sooty_Albatross] 第 7 页 (offset=300)
✅ 第 7 页新增 36 张，累计 280 张。
📄 [Sooty_Albatross] 第 8 页 (offset=350)
✅ 第 8 页新增 31 张，累计 311 张。
📄 [Sooty_Albatross] 第 9 页 (offset=400)
✅ 第 9 页新增 35 张，累计 346 张。
📄 [Sooty_Albatross] 第 10 页 (offset=450)
✅ 第 10 页新增 23 张，累计 369 张。
📄 [Sooty_Albatross] 第 11 页 (offset=500)
✅ 第 11 页新增 15 张，累计 384 张。
📄 [Sooty_Albatross] 第 12 页 (offset=550)
❌ 没有更多图片，提前结束。
📸 [Sooty_Albatross] 共抓取 384 张（目标 600）



...

✅ Rhinoceros_Auklet 下载完成，共 303 张图片。
🦉 开始爬取：Brewer_Blackbird
📄 [Brewer_Blackbird] 第 1 页 (offset=0)
✅ 第 1 页新增 29 张，累计 29 张。
📄 [Brewer_Blackbird] 第 2 页 (offset=50)
✅ 第 2 页新增 33 张，累计 62 张。
📄 [Brewer_Blackbird] 第 3 页 (offset=100)
✅ 第 3 页新增 33 张，累计 95 张。
📄 [Brewer_Blackbird] 第 4 页 (offset=150)
✅ 第 4 页新增 35 张，累计 130 张。
📄 [Brewer_Blackbird] 第 5 页 (offset=200)
✅ 第 5 页新增 32 张，累计 162 张。
📄 [Brewer_Blackbird] 第 6 页 (offset=250)
✅ 第 6 页新增 34 张，累计 196 张。
📄 [Brewer_Blackbird] 第 7 页 (offset=300)
✅ 第 7 页新增 34 张，累计 230 张。
📄 [Brewer_Blackbird] 第 8 页 (offset=350)
✅ 第 8 页新增 31 张，累计 261 张。
📄 [Brewer_Blackbird] 第 9 页 (offset=400)
✅ 第 9 页新增 33 张，累计 294 张。
📄 [Brewer_Blackbird] 第 10 页 (offset=450)
✅ 第 10 页新增 13 张，累计 307 张。
📄 [Brewer_Blackbird] 第 11 页 (offset=500)
✅ 第 11 页新增 2 张，累计 309 张。
📄 [Brewer_Blackbird] 第 12 页 (offset=550)
❌ 没有更多图片，提前结束。
📸 [Brewer_Blackbird] 共抓取 309 张（目标 600）



📥 Brewer_Blackbird: 100%|██████████████████████████████████████████████████████████| 309/309 [00:49<00:00,  6.19img/s]


✅ Brewer_Blackbird 下载完成，共 309 张图片。
🦉 开始爬取：Red_winged_Blackbird
📄 [Red_winged_Blackbird] 第 1 页 (offset=0)
✅ 第 1 页新增 32 张，累计 32 张。
📄 [Red_winged_Blackbird] 第 2 页 (offset=50)
✅ 第 2 页新增 30 张，累计 62 张。
📄 [Red_winged_Blackbird] 第 3 页 (offset=100)
✅ 第 3 页新增 30 张，累计 92 张。
📄 [Red_winged_Blackbird] 第 4 页 (offset=150)
✅ 第 4 页新增 29 张，累计 121 张。
📄 [Red_winged_Blackbird] 第 5 页 (offset=200)
✅ 第 5 页新增 31 张，累计 152 张。
📄 [Red_winged_Blackbird] 第 6 页 (offset=250)
✅ 第 6 页新增 32 张，累计 184 张。
📄 [Red_winged_Blackbird] 第 7 页 (offset=300)
✅ 第 7 页新增 35 张，累计 219 张。
📄 [Red_winged_Blackbird] 第 8 页 (offset=350)
✅ 第 8 页新增 33 张，累计 252 张。
📄 [Red_winged_Blackbird] 第 9 页 (offset=400)
✅ 第 9 页新增 33 张，累计 285 张。
📄 [Red_winged_Blackbird] 第 10 页 (offset=450)
✅ 第 10 页新增 28 张，累计 313 张。
📄 [Red_winged_Blackbird] 第 11 页 (offset=500)
✅ 第 11 页新增 21 张，累计 334 张。
📄 [Red_winged_Blackbird] 第 12 页 (offset=550)
❌ 没有更多图片，提前结束。
📸 [Red_winged_Blackbird] 共抓取 334 张（目标 600）



📥 Red_winged_Blackbird: 100%|██████████████████████████████████████████████████████| 334/334 [00:58<00:00,  5.75img/s]

✅ Red_winged_Blackbird 下载完成，共 334 张图片。

🎉 所有鸟类下载完成！总耗时 689.81 秒
